In [ ]:
import pandas as pd
import obspy
import matplotlib.pyplot as plt
from datetime import timezone

import numpy as np
from PIL import Image

In [ ]:
fin_dq = pd.read_csv(
    '/network/projects/aia/whale_call/DATA_QUALITY/blue/blue_quality.csv',
    header=0
    )

In [ ]:
ds = pd.read_csv('/network/projects/aia/whale_call/LABELS/BW/bw_raw.csv')

# Plot matching WAV and Spectrograms

In [ ]:
for index, row in fin_dq.iterrows():

    date = row['date']
    station = row['station']
    component = row['component']

    datetime_start = date + " " + row['t_start']
    datetime_end = date + " " + row['t_end']
    print("=========================== Start : {} | End : {} ===========================".format(datetime_start,datetime_end))
    print("Index:",index)
    print("Station:",station)
    print("date:",date)
    print("component:",component)


    folder = date[:4]+date[5:7]+date[8:10]
    file = date[:4]+"."+date[5:7]+"."+date[8:10]+".CN."+station+".."+component+".SAC"
    
    st = obspy.read(
        '/network/projects/aia/whale_call/FILT_10_32/'+folder+"/"+file,
    )
    start_time=obspy.UTCDateTime(datetime_start)
    end_time=obspy.UTCDateTime(datetime_end)

    st_1 = st.slice(
        start_time, 
        end_time
        )

    tr = st_1[0]

    img = "/network/projects/aia/whale_call/DATA_QUALITY/"+row['whale_type']+"/solo/"+str(index+1)+".png"


    dss =  ds[(pd.to_datetime(ds.time_call_start) > pd.to_datetime(datetime_start)) & (pd.to_datetime(ds.time_call_start) < pd.to_datetime(datetime_end)) & (ds.component == component) & (ds.station_code == station)].copy()

    dss['time_call_start_dt'] = pd.to_datetime(dss.time_call_start)
    dss['time_call_end_dt'] = pd.to_datetime(dss.time_call_end)

    dss['datetime_start_tz'] = dss.time_call_start_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))
    dss['datetime_end_tz'] = dss.time_call_end_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))

    # Create a figure and axes object
    fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
    ax[0].autoscale(tight=True)

    # Plot the cosine function
    ax[0].plot(tr.times("matplotlib"), tr.data, "b-")


    if dss.shape[0] > 0:
    # Plot whales detections on timeline
        for index,row in dss.iterrows():
            ax[0].axvline(
                x=row['datetime_start_tz'],
                color='g',
                label="whale Call")
            
            ax[0].axvline(
                x=row['datetime_end_tz'],
                color='r',
                label="whale Call")

    ax[0].set_title('Filtered SAC')

    # Load and show the image
    image = Image.open(img)
    ax[1].imshow(image)
    ax[1].set_title('Spectrogram')

    # Adjust the layout
    plt.tight_layout()

    # Display the plot
    plt.show()     


# Compare individual calls

In [ ]:
for index, row in fin_dq.loc[[4]].iterrows():

    date = row['date']
    station = row['station']
    component = row['component']

    datetime_start = date + " " + row['t_start']
    datetime_end = date + " " + row['t_end']
    print("Start : {} | End : {}".format(datetime_start,datetime_end))


    folder = date[:4]+date[5:7]+date[8:10]
    file = date[:4]+"."+date[5:7]+"."+date[8:10]+".CN."+station+".."+component+".SAC"
    
    st = obspy.read(
        '/network/projects/aia/whale_call/FILT_10_32/'+folder+"/"+file,
    )
    start_time=obspy.UTCDateTime(datetime_start)
    end_time=obspy.UTCDateTime(datetime_end)

    st_1 = st.slice(
        start_time, 
        end_time
        )

    tr = st_1[0]
    total_npts = tr.stats.npts

    img = "/network/projects/aia/whale_call/DATA_QUALITY/"+row['whale_type']+"/solo/"+str(index+1)+".png"
    print(datetime_start)
    print(datetime_end)

    dss =  ds[(pd.to_datetime(ds.time_call_start) > pd.to_datetime(datetime_start)) & (pd.to_datetime(ds.time_call_start) < pd.to_datetime(datetime_end)) & (ds.component == component) & (ds.station_code == station)].copy()


    dss['time_call_start_dt'] = pd.to_datetime(dss.time_call_start)
    dss['time_call_end_dt'] = pd.to_datetime(dss.time_call_end)

    dss['datetime_start_tz'] = dss.time_call_start_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))
    dss['datetime_end_tz'] = dss.time_call_end_dt.apply(lambda x : x.replace(tzinfo=timezone.utc))

    # Create a figure and axes object
    fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
    ax[0].autoscale(tight=True)

    # Plot the cosine function
    ax[0].plot(tr.times("matplotlib"), tr.data, "b-")

    if dss.shape[0] > 0:
    # Plot whales detections on timeline
        for index,row in dss.iterrows():
            ax[0].axvline(
                x=row['datetime_start_tz'],
                color='g',
                label="whale Call")
            
            ax[0].axvline(
                x=row['datetime_end_tz'],
                color='r',
                label="whale Call")

    ax[0].set_title('Filtered SAC')

    # Load and show the image
    image = Image.open(img)
    ax[1].imshow(image)
    ax[1].set_title('Spectrogram')

    # Adjust the layout
    plt.tight_layout()

    # Display the plot
    plt.show()

    for index,row in dss.iloc[2:,:].iterrows():

        st_2 = st.slice(
            obspy.UTCDateTime(row['time_window_start']),
            obspy.UTCDateTime(row['time_window_end'])
        )
        tr = st_2[0]

        # Create a figure and axes object
        fig, ax = plt.subplots(nrows=2, figsize=(8, 8))
        ax[0].autoscale(tight=True)

        # Plot the cosine function
        ax[0].plot(tr.times("matplotlib"), tr.data, "b-")

        ax[0].axvline(
            x=row['datetime_start_tz'],
            color='g',
            label="whale Call")
        
        ax[0].axvline(
            x=row['datetime_end_tz'],
            color='r',
            label="whale Call")

        from PIL import Image
        import numpy as np
        
        # Load image
        image = Image.open(img)
        # Convert to np array
        image_arr = np.array(image)

        # dimensions
        hauteur = image_arr.shape[0]
        longueur = image_arr.shape[1]

        total_wav = len(tr.data)

        from datetime import datetime
        d1 = datetime.strptime(datetime_start, "%Y-%m-%d %H:%M:%S")
        d2 = datetime.strptime(datetime_end, "%Y-%m-%d %H:%M:%S")
        d3 = datetime.strptime(row['time_window_start'], "%Y-%m-%d %H:%M:%S.%f")
        d4 = datetime.strptime(row['time_window_end'], "%Y-%m-%d %H:%M:%S.%f")

        # difference between dates in timedelta
        diff_start = ((d3-d1).total_seconds())
        diff_end = ((d4-d1).total_seconds())
        second_block = abs(longueur / (d1 - d2).total_seconds())

        # Crop image
        image_arr = image_arr[
            :, # Hauteur
            round(diff_start*second_block):round(diff_end*second_block) # Longueur
        ]

        # Convert array to image
        image = Image.fromarray(image_arr)
        
        # Display image
        # ax[1].imshow(image)
        # plt.show()

        new_image = image.resize((1500, 400))
        # new_image.save('image_400.jpg')
        ax[1].imshow(new_image) 
        plt.show()

# Check if data exists

In [ ]:
import scipy
import pandas as pd

In [ ]:
# Load .mat file
mat = scipy.io.loadmat(
    "/network/projects/aia/whale_call/MATLAB_OUTPUT/WhaleDetectionsLSZ_new.mat", 
    variable_names=["FWC", "BWC", "stadir", "stadir_bw"]
)

In [ ]:
# Load Fin & Blue whale calls from .mat
colnames_WC = [
    "Datenum",
    "station_number",
    "R",
    "SNR",
    "group_index",
    "time_start",
    "num_calls_in_detection",
    "detection_id",
]

In [ ]:
def preprocess_(dataset: pd.DataFrame, stadir: dict) -> pd.DataFrame:
    """Preprocess labels

    @param: dataset (pd.DataFrame): labels
    @param: stadir (dict): name and index of stations

    return: pd.Dataframe preprocessed labels
    """

    # Time
    dataset["datetime"] = pd.to_datetime(dataset["Datenum"] - 719529, unit="D")
    dataset["time"] = dataset["datetime"].dt.time
    dataset["date"] = dataset["datetime"].dt.date
    dataset["datetime_start"] = pd.to_datetime(
        dataset["time_start"].round(), unit="s"
    ).dt.time

    # Convert column types
    integer_columns = [
        "num_calls_in_detection",
        "group_index",
        "station_number",
        "detection_id",
    ]
    dataset[integer_columns] = dataset[integer_columns].astype(int)

    # Station number
    dataset["station_name"] = dataset["station_number"].map(stadir)

    return dataset

In [ ]:
def read_stadir(mat_file: dict, whale_type: str) -> dict:
    """
    Return name of all stations stored in stadir var
    in the .mat file

    @param: mat_file (String): path + name of .mat file
    @param: whale_type (String): type of whales

    return: dict containing station names with index as key
    and value as station name
    """
    list_stations = [
        name_station[0] for name_station in mat_file[whale_type][:, 0]
    ]
    return {v + 1: k for v, k in enumerate(list_stations)}

In [ ]:
# Get data from matlab .mat matrix
df_calls = pd.DataFrame(
    mat["bw".upper() + "C"], columns=colnames_WC
)

In [ ]:
# Load name and index of stations
stadir = read_stadir(
    mat, "stadir_bw"
)

# Preprocess labels
labels = preprocess_(df_calls, stadir)

In [ ]:
date_start = "2021-03-05 5:34:00"
date_end = "2021-03-05 5:46:00"

In [ ]:
ds[(pd.to_datetime(ds.time_call_start) > pd.to_datetime(date_start)) & (pd.to_datetime(ds.time_call_start) < pd.to_datetime(date_end)) & (ds.component == component) & (ds.station_code == station)]

In [ ]:
labels[(labels.datetime > pd.to_datetime(date_start)) & (labels.datetime < pd.to_datetime(date_end))]